Username: dscc440bot
Password: si.VbFxBHy559Jn

Documentation:
https://praw.readthedocs.io/en/latest/index.html

In [1]:
# Create read-only reddit instance
import praw

# This should be moved to a private praw.ini file at some point
reddit = praw.Reddit(
 client_id="kuf-S39ElN0SpA",
 client_secret="XVSM3GfyGRbhbdeXh-s3AtNqnP2FCg",
 user_agent="pc:attributes.from.punctuation:v1.0.0 (by u/dscc440bot)"
)

In [65]:
###Main method
import requests
import csv
from datetime import datetime
import time

year_utc = 31536000  # UTC time for one year
start_time = 1607162934  # Used as the reference start time for comments and age adjustment

#list of subreddits with demographic-rich submissions
subreddit_names = [
    "relationships",
    "relationship_advice",
    "RoastMe",
    "RateMe",
    "amiugly",
    "AmItheAsshole",
    "Needafriend",
    "Selfie"
]

#api request
payload = {'subreddit': None, 'sort': 'desc', 'sort_type': 'created_utc', 'before': None, 'size': '500'}

#initialize output csv
keys = ['age', 'gender', 'author', 'full_link']
#with open('stats.csv', 'w', newline='') as output_file:
#    dict_writer = csv.DictWriter(output_file, keys)
#    dict_writer.writeheader()

#iterate through subreddits with demographic-rich post titles
for sub_name in subreddit_names:
    payload['subreddit'] = sub_name
    print(f"Collecting from r/{sub_name}")
    batch_count = 0
    
    #iterate through subreddit in batches
    while True:
        dataset = []  #initialize dataset (list of dictionaries), resets each loop after batch is saved to csv
        payload['before'] = str(start_time)
        print(payload)
        r = requests.get('https://api.pushshift.io/reddit/search/submission/', params=payload)
        r.encoding = 'utf-8'
        time.sleep(1)
        try:
            json = r.json()
        except Exception:
            print("Failed to parse:")
            print(r.content)
            
        objects = json['data']
        if len(objects) == 0:  # if response was empty, end loop
            break
        for subm in objects:
            if subm['author'] != '[deleted]':  # make sure there is a usable author
                entry = find_demographics(subm['title'] + " " + subm['selftext'])
                if entry is not None:  #if there are demographics to use
                    #Calculate current age
                    if entry['age'] is not None:
                        utc_at_comment = subm['created_utc']
                        years_since = int((start_time - utc_at_comment)/year_utc)
                        entry['age'] = entry['age'] + years_since
                    # Prevent error while writing to csv if link contains unicode character
                    try:
                        subm['full_link'].encode('ascii')
                    except UnicodeEncodeError:
                        print(f"Skipping {subm['full_link']}")
                        continue
                    entry['full_link'] = subm['full_link']
                    entry['author'] = subm['author']
                    dataset.append(entry)
        
        #Save dataset to csv
        if len(dataset) > 0:
            with open('stats.csv', 'a', newline='') as output_file:
                dict_writer = csv.DictWriter(output_file, keys)
                dict_writer.writerows(dataset)
        #Cleanup for next batch
        batch_count += 1
        print(f"Finished Batch {batch_count}")
        start_time = objects[-1]['created_utc']-1

{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1607162934', 'size': '500'}
Finished Batch 1
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1607156068', 'size': '500'}
Finished Batch 2
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1607150904', 'size': '500'}
Finished Batch 3
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1607146366', 'size': '500'}
Skipping https://www.reddit.com/r/relationship_advice/comments/k71c1t/how_can_i_nicely_ask_my_23f_fiancé_25m_to_leave/
Finished Batch 4
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1607141560', 'size': '500'}
Skipping https://www.reddit.com/r/relationship_advice/comments/k6zyts/how_can_i_ask_my_23f_fiancé_25m_to_leave_the/
Finished Batch 5
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc',

Skipping https://www.reddit.com/r/relationship_advice/comments/k6e651/please_help_me_34f_fiancé_32m/
Finished Batch 22
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1607057132', 'size': '500'}
Finished Batch 23
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1607051477', 'size': '500'}
Finished Batch 24
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1607011951', 'size': '500'}
Skipping https://www.reddit.com/r/relationship_advice/comments/k600iq/should_i_24f_go_away_and_leave_my_fiancé_25m_alone/
Finished Batch 25
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1607005782', 'size': '500'}
Finished Batch 26
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1606998685', 'size': '500'}
Finished Batch 27
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'cr

Finished Batch 46
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1606915836', 'size': '500'}
Skipping https://www.reddit.com/r/relationship_advice/comments/k594dl/my_22f_fiancée_24m_cheated_with_a_transsexual/
Skipping https://www.reddit.com/r/relationship_advice/comments/k58p28/my_34f_fiancé_37m_cares_for_his_exs_daughter_and/
Skipping https://www.reddit.com/r/relationship_advice/comments/k5804e/my26m_fiancée25f_are_running_out_of_things_to_do/
Finished Batch 47
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1606909641', 'size': '500'}
Finished Batch 48
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1606902627', 'size': '500'}
Finished Batch 49
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1606895560', 'size': '500'}
Finished Batch 50
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'create

Skipping https://www.reddit.com/r/relationship_advice/comments/k4b317/fiancée_31f_once_let_an_ex_film_them_have_sex/
Finished Batch 74
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1606786190', 'size': '500'}
Finished Batch 75
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1606782775', 'size': '500'}
Finished Batch 76
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1606779237', 'size': '500'}
Skipping https://www.reddit.com/r/relationship_advice/comments/k487k4/i_20m_broke_up_with_my_fiancé_19f_due_to_her/
Finished Batch 77
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1606776505', 'size': '500'}
Finished Batch 78
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1606772843', 'size': '500'}
Finished Batch 79
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_

Finished Batch 90
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1606722037', 'size': '500'}
Skipping https://www.reddit.com/r/relationship_advice/comments/k3qxiv/my_fiancé_has_been_on_an_adult_chat_site_how_do/
Finished Batch 91
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1606717675', 'size': '500'}
Skipping https://www.reddit.com/r/relationship_advice/comments/k3pn2q/my_fiancé_left_me_and_im_not_sure_how_to_handle_it/
Finished Batch 92
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1606713704', 'size': '500'}
Finished Batch 93
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1606709223', 'size': '500'}
Skipping https://www.reddit.com/r/relationship_advice/comments/k3ntck/my_24m_fiancé_22f_makes_me_sad_yet_i_love_her_so/
Finished Batch 94
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'cr

...

Finished Batch 975
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1602663372', 'size': '500'}
Finished Batch 976
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1602657735', 'size': '500'}
Finished Batch 977
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1602653297', 'size': '500'}
Finished Batch 978
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1602649088', 'size': '500'}
Failed to parse:
b'<!DOCTYPE html>\n<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->\n<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->\n<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->\n<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->\n<head>\n<meta http-equiv="refresh" content="0">\n\n<title>api.pushshift.io | 522: Connection timed out</t

Skipping https://www.reddit.com/r/relationship_advice/comments/jansps/my_31m_fiancée_32f_wants_to_adopt_her_sisters_5/
Finished Batch 986
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1602624198', 'size': '500'}
Skipping https://www.reddit.com/r/relationship_advice/comments/jam79r/my_24f_fiancée_24m_has_turned_into_a_total/
Finished Batch 987
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1602620224', 'size': '500'}
Finished Batch 988
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1602617106', 'size': '500'}
Skipping https://www.reddit.com/r/relationship_advice/comments/jajgfq/how_do_i_27m_go_platonic_with_former_fiancé_25/
Finished Batch 989
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1602614031', 'size': '500'}
Finished Batch 990
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created

Finished Batch 995
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1602595888', 'size': '500'}
Finished Batch 996
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1602591364', 'size': '500'}
Finished Batch 997
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1602584889', 'size': '500'}
Finished Batch 998
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1602577883', 'size': '500'}
Finished Batch 999
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1602573050', 'size': '500'}
Finished Batch 1000
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1602567668', 'size': '500'}
Skipping https://www.reddit.com/r/relationship_advice/comments/ja7694/my_28m_ex_fiancée_25f_wont_leave_me_alone/
Skipping https://www.reddit.com/r/relationship_advic

Skipping https://www.reddit.com/r/relationship_advice/comments/j90w6w/fiancé_lied_about_deleting_snapchat/
Finished Batch 1040
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1602399614', 'size': '500'}
Finished Batch 1041
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1602395580', 'size': '500'}
Finished Batch 1042
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1602390421', 'size': '500'}
Finished Batch 1043
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1602386623', 'size': '500'}
Finished Batch 1044
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1602382167', 'size': '500'}
Finished Batch 1045
{'subreddit': 'relationship_advice', 'sort': 'desc', 'sort_type': 'created_utc', 'before': '1602378130', 'size': '500'}
Finished Batch 1046
{'subreddit': 'relationship_advic

PermissionError: [Errno 13] Permission denied: 'stats.csv'

In [3]:
import requests
import csv
from datetime import datetime
import time

#csv cleanup (Remove duplicates, remove inactive or throwaway accounts)
week_utc = 604800

comments_per_week = 0.5 #comments expected per month
ref_time = 1607162934  # starting time
weeks_before = 24 
total_count = 0
dq_count = 0

start_time = ref_time
end_time = ref_time - (week_utc*weeks_before)
thresh = comments_per_week*weeks_before

name_check = {}

in_filename = "stats.csv"
out_filename = "stats_clean.csv"

with open(in_filename, "r") as in_file:
    reader = csv.DictReader(in_file)
    size = len(list(reader))
    print("Comments to parse: " + str(size))

with open(in_filename, "r") as in_file, open(out_filename, "w") as out_file:
    reader = csv.DictReader(in_file)
    keys = ['age', 'gender', 'author', 'full_link', 'text']
    writer = csv.DictWriter(out_file, keys)
    writer.writeheader()
    
    for row in reader:
        total_count += 1
        if total_count % int(size/1000) == 0:
            print(f"{int(total_count/size*100)}% Complete. Currently, {dq_count} rows have been removed (out of {total_count})")
        
        #check for duplicate
        if row['author'] in name_check:
            dq_count += 1
            continue
        else:
            name_check[row['author']] = None
            
        #check for inactive accounts
        comment_list = []  # contains strings of comment text
        redditor = reddit.redditor(row['author'])
        try:
            for comment in redditor.comments.new(limit=None):
                if comment.created_utc < start_time:  # Check if comment is within time range
                    if comment.created_utc < end_time:
                        break
                    else:
                        if comment.created_utc > end_time:
                            comment_list.append(comment.body)
        except:
            #print("Account not valid (probably suspended)[" + redditor.name + "]")

        if len(comment_list) < thresh:
            dq_count += 1
            continue
        else:
            #generate text
            sample = '@ '.join(comment_list)
            sample = sample.encode("ascii", "ignore").decode()  # Clean to ASCII
            sample = sample.replace('\n', " ")  # Remove new line
            row['text'] = sample
        #map to output csv
        writer.writerow(row)

Comments to parse: 116978
Account not valid (probably suspended)[maxleykittiver]
0% Complete. Currently, 66 rows have been removed (out of 116)
Account not valid (probably suspended)[Relationship-Noobie2]
Account not valid (probably suspended)[TenYearsTogether]
Account not valid (probably suspended)[calcifer100000000009]
0% Complete. Currently, 138 rows have been removed (out of 232)
Account not valid (probably suspended)[ThrowRAsameone]
0% Complete. Currently, 217 rows have been removed (out of 348)
Account not valid (probably suspended)[Ok-Potato8139]
0% Complete. Currently, 292 rows have been removed (out of 464)
Account not valid (probably suspended)[pinkclouddss]
Account not valid (probably suspended)[redroses46]
0% Complete. Currently, 379 rows have been removed (out of 580)
Account not valid (probably suspended)[GeorgeB2000]
Account not valid (probably suspended)[ReleaseVisible7163]
Account not valid (probably suspended)[throwRA0226]
Account not valid (probably suspended)[Workin

3% Complete. Currently, 2521 rows have been removed (out of 3596)
Account not valid (probably suspended)[Particular_Act_6299]
Account not valid (probably suspended)[dreamintaiwanese]
Account not valid (probably suspended)[throwaway1294509]
Account not valid (probably suspended)[throwRAbeansfolunch]
Account not valid (probably suspended)[banana_oatmeal124]
Account not valid (probably suspended)[MomentofMeat]
3% Complete. Currently, 2599 rows have been removed (out of 3712)
Account not valid (probably suspended)[WhitWhit00]
Account not valid (probably suspended)[MartinMassi87]
3% Complete. Currently, 2686 rows have been removed (out of 3828)
Account not valid (probably suspended)[Monkey_Shit-]
Account not valid (probably suspended)[Dear-Cauliflower2683]
Account not valid (probably suspended)[Birdsandwildlife]
Account not valid (probably suspended)[ifbsklsber]
3% Complete. Currently, 2772 rows have been removed (out of 3944)
Account not valid (probably suspended)[ChallengeClassic9872]
Acc

Account not valid (probably suspended)[throwaway138115]
Account not valid (probably suspended)[creepyoroverthink]
5% Complete. Currently, 4745 rows have been removed (out of 6728)
Account not valid (probably suspended)[ThrowRA746194]
Account not valid (probably suspended)[acatnamedcricket]
Account not valid (probably suspended)[lionjourney]
Account not valid (probably suspended)[plshelpme121212]
Account not valid (probably suspended)[elekieos]
Account not valid (probably suspended)[__taurus]
Account not valid (probably suspended)[PermacultureLover]
Account not valid (probably suspended)[medium-good]
5% Complete. Currently, 4827 rows have been removed (out of 6844)
Account not valid (probably suspended)[-daddy--issues-]
5% Complete. Currently, 4910 rows have been removed (out of 6960)
Account not valid (probably suspended)[save_drama_for_mama]
Account not valid (probably suspended)[RAL0178]
Account not valid (probably suspended)[TownFuzzy]
Account not valid (probably suspended)[throwra6

Account not valid (probably suspended)[pizzaboyweedsmoker]
Account not valid (probably suspended)[Ok-Function-2994]
Account not valid (probably suspended)[Heavendonthell]
Account not valid (probably suspended)[crunch-buttsteak]
Account not valid (probably suspended)[Ball_Balls]
8% Complete. Currently, 6659 rows have been removed (out of 9396)
Account not valid (probably suspended)[iloveitalianmen22]
Account not valid (probably suspended)[sheskillingmesoftly]
Account not valid (probably suspended)[pizzaaa_pizzaaa]
Account not valid (probably suspended)[throwawaynew30]
Account not valid (probably suspended)[Slacky101]
8% Complete. Currently, 6746 rows have been removed (out of 9512)
Account not valid (probably suspended)[skywarriorpy]
Account not valid (probably suspended)[Friendly_Lettuce_878]
Account not valid (probably suspended)[TingGoBrrrrrrrrrr]
Account not valid (probably suspended)[AnyImpression5282]
Account not valid (probably suspended)[Apple_Nuts--]
Account not valid (probably

Account not valid (probably suspended)[ThrowRAaaaargh]
10% Complete. Currently, 8372 rows have been removed (out of 11716)
Account not valid (probably suspended)[OkStud99]
Account not valid (probably suspended)[sjkdje]
Account not valid (probably suspended)[akkamajsjsnsm]
Account not valid (probably suspended)[BlastuWithMy40]
Account not valid (probably suspended)[throwawaywoohoop]
Account not valid (probably suspended)[throwaway2246187478]
10% Complete. Currently, 8458 rows have been removed (out of 11832)
Account not valid (probably suspended)[dater101]
Account not valid (probably suspended)[hdsh55226]
Account not valid (probably suspended)[Nattt01]
Account not valid (probably suspended)[unwantedchild7482]
Account not valid (probably suspended)[Squarehead__]
Account not valid (probably suspended)[theseaishistory]
10% Complete. Currently, 8535 rows have been removed (out of 11948)
Account not valid (probably suspended)[_Frogggy_]
Account not valid (probably suspended)[TexanRedditor2]


...

Account not valid (probably suspended)[ThrowRAParentsIssue]
Account not valid (probably suspended)[throwra8282826]
Account not valid (probably suspended)[Throw-RA34543]
Account not valid (probably suspended)[glassnpatron]
Account not valid (probably suspended)[ThrowRA-0909244]
90% Complete. Currently, 81726 rows have been removed (out of 105444)
Account not valid (probably suspended)[ilovemyteacher194]
Account not valid (probably suspended)[throwaway263845]
Account not valid (probably suspended)[throwra41239129319]
Account not valid (probably suspended)[mo0chies]
Account not valid (probably suspended)[ineedhelp4958]
90% Complete. Currently, 81822 rows have been removed (out of 105560)
Account not valid (probably suspended)[ThrowRA-qkwjwnw]
Account not valid (probably suspended)[grass-Animals]
Account not valid (probably suspended)[varioray]
Account not valid (probably suspended)[throwra82827153]
90% Complete. Currently, 81920 rows have been removed (out of 105676)
Account not valid (pr

92% Complete. Currently, 83556 rows have been removed (out of 107880)
Account not valid (probably suspended)[throwra2355]
Account not valid (probably suspended)[nonailbiter]
Account not valid (probably suspended)[ImgurIsBetter23]
Account not valid (probably suspended)[Square_Nutlol]
Account not valid (probably suspended)[BlaqKeen]
Account not valid (probably suspended)[ThrowRA-LonelyPumpki]
Account not valid (probably suspended)[bleebljop99]
92% Complete. Currently, 83638 rows have been removed (out of 107996)
Account not valid (probably suspended)[PassionDense598]
Account not valid (probably suspended)[HowlingMad2001]
Account not valid (probably suspended)[avatarkorean]
92% Complete. Currently, 83724 rows have been removed (out of 108112)
Account not valid (probably suspended)[Reckless_69]
Account not valid (probably suspended)[ThrowRa-174]
Account not valid (probably suspended)[idkmysister]
Account not valid (probably suspended)[shawnab22]
Account not valid (probably suspended)[Nut_J

Account not valid (probably suspended)[Mcboujee]
Account not valid (probably suspended)[Dancethenightawaty2]
Account not valid (probably suspended)[ThrowRA_gameboy_23]
Account not valid (probably suspended)[reojsdfoj]
Account not valid (probably suspended)[ThrowRA_brosandhoes]
Account not valid (probably suspended)[Obvious_Lime_4864]
94% Complete. Currently, 85473 rows have been removed (out of 110432)
Account not valid (probably suspended)[throwaway12314325]
Account not valid (probably suspended)[babycornfalala]
Account not valid (probably suspended)[ThrowawayRA5068]
Account not valid (probably suspended)[throwaway50685068]
Account not valid (probably suspended)[_nerdychemist]
Account not valid (probably suspended)[HesrshedYkI]
94% Complete. Currently, 85557 rows have been removed (out of 110548)
Account not valid (probably suspended)[ThrowRA39253]
Account not valid (probably suspended)[NoAd6150huji]
Account not valid (probably suspended)[Throw2495away]
Account not valid (probably sus

Account not valid (probably suspended)[edgybutok]
Account not valid (probably suspended)[PumperNickel0]
Account not valid (probably suspended)[ThrowRaboomergrandpa]
Account not valid (probably suspended)[chnkn_nungt]
96% Complete. Currently, 87229 rows have been removed (out of 112752)
Account not valid (probably suspended)[teenager_throw12345]
Account not valid (probably suspended)[Ziggy_Lopper2020]
Account not valid (probably suspended)[rickshaw23232]
Account not valid (probably suspended)[throwRAlandlordbf]
96% Complete. Currently, 87313 rows have been removed (out of 112868)
Account not valid (probably suspended)[TheGaminator802]
96% Complete. Currently, 87398 rows have been removed (out of 112984)
Account not valid (probably suspended)[ThrowRAconfusedtrash]
Account not valid (probably suspended)[Unfathomable_God]
Account not valid (probably suspended)[Defiant_Wish5500]
Account not valid (probably suspended)[ThrowRA10061995Tim]
Account not valid (probably suspended)[vsadfairy]
Acco

98% Complete. Currently, 89028 rows have been removed (out of 115188)
Account not valid (probably suspended)[ThrowRA_MeTime]
Account not valid (probably suspended)[gottalovereddit182]
Account not valid (probably suspended)[Macaroon_Soft]
Account not valid (probably suspended)[ThrowRA0756]
98% Complete. Currently, 89107 rows have been removed (out of 115304)
Account not valid (probably suspended)[ThrowRA_wiwk]
Account not valid (probably suspended)[HotApricot3]
Account not valid (probably suspended)[ThrowRA_0610]
98% Complete. Currently, 89192 rows have been removed (out of 115420)
Account not valid (probably suspended)[HotCoffee216]
Account not valid (probably suspended)[gglitchbbitch]
Account not valid (probably suspended)[InterestingRegular29]
Account not valid (probably suspended)[side_dude]
Account not valid (probably suspended)[venus1ove]
Account not valid (probably suspended)[throwaway2728282727]
Account not valid (probably suspended)[ListedOns]
98% Complete. Currently, 89288 row

In [3]:
#csv cleanup (Remove duplicates, remove inactive or throwaway accounts)
week_utc = 604800

comments_per_week = 1 #comments expected per month
ref_time = 1607162934  # starting time
weeks_before = 24 
total_count = 0
dq_count = 0

start_time = ref_time
end_time = ref_time - (week_utc*weeks_before)
thresh = comments_per_week*weeks_before

#check for inactive accounts
comment_list = []  # contains strings of comment text
redditor = reddit.redditor('stakstak')
try:
    for comment in redditor.comments.new(limit=None):
        if comment.created_utc < start_time:  # Check if comment is within time range
            if comment.created_utc < end_time:
                break
            else:
                if comment.created_utc > end_time:
                    comment_list.append(comment.body)
except:
    print("Account not valid (probably suspended).")

In [2]:
year_utc = 31536000

def generate_data_entry(text_instance, current_utc):
    '''
    Generate an entry to be added to the final dataset. Entry will be created from text_instance object. Text_instance can be either a submision or comment object.
    
    Returns dictionary(?) containing redditor name, age, gender, and punctuation statistics
    Returns None if no useful data entry can be generated.
    '''
    # Check if author's account still exists
    if text_instance.author is None:
        return None
    
    text = None
    if type(text_instance) is praw.models.Submission:
        text = text_instance.title + text_instance.selftext
    elif type(text_instance) is praw.models.reddit.comment.Comment:
        text = text_instance.body
    else:
        #error
        print("Item cannot be parsed (Is not a comment or submission)")
        print(type(text_instance))

    #Find demographics
    demog = find_demographics(text)
    
    if demog is None:
        return None
    
    #Calculate current age
    if demog["age"] is not None:
        utc_at_comment = text_instance.created_utc
        years_since = int((current_utc - utc_at_comment)/year_utc)
        demog["age"] = demog["age"] + years_since
    
    # For testing, make each entity row identifiable
    demog["url"] = text_instance.permalink
    
    #If demographics are found, calculate puncuation statistics
    p_stats = calculate_punctuation_stats(text_instance.author,1588291200,1604188800)  #05/01/2020-11/01/2020
    
    if p_stats is None:
        return None
    
    #Format and return data entry
    return {**demog, **p_stats}

In [2]:
import re

def find_demographics(text):
    '''
    Find the age and gender information contained in the text parameter.
    
    Returns a dictionary of the age and gender information contained in the text. If a feature cannot be determined, its dictionary value will be None.
    '''
    out = {}
    out["age"] = None
    out["gender"] = None
    
    ### find demographic info
    # Check for ##Gender format with or without space or coma (Ex. 32M or 28 F or 23,M or 54, F)
    match = re.findall(r"\b(\d{2}),? ?\/?(M|m|Male|male|F|f|Female|female)\b",text)
    if len(match) == 1:
        out["age"] = int(match[0][0])
        g = match[0][1]
        if g in "F f Female female":
            out["gender"] = "Female"
        else:
            out["gender"] = "Male"
        return out
    elif len(match) > 1:  # Catches cases where two people are identified (Ex. My wife (23f) and I (25M))
        # first, focus matched text to more specific criteria including identifying pronoun
        match = re.findall(r"(?:I|i|me|Me|my|My|I'[a-zA-Z]+|i'[a-zA-Z]+)\s?[\(\[\s](\d{2})[,\/ ]?(M|m|Male|male|F|f|Female|female)[\)\]\s]",text)
        if match:
            out["age"] = int(match[0][0])
            g = match[0][1]
            if g in "F f Female female":
                out["gender"] = "Female"
            else:
                out["gender"] = "Male"
            return out
    
    return None

In [5]:
import re
import itertools
import pandas as pd
from statistics import mean
from string import punctuation
from itertools import chain
import math

thresh = 10  # Threshold for acceptable comment. To be acceptable, must have thresh comments and punctuation marks used.

def calculate_punctuation_stats(redditor, start_time, end_time):
    '''
    Generate punctuation statistics from posts of redditor during the time range between start_time and end_time (both in UTC, Unix time).
    
    Returns dictionary(?) of punctuation statistics
    '''
    comment_list = []  # contains strings of comment text
    
    try:
        for comment in redditor.comments.new(limit=None):
            if comment.created_utc > start_time and comment.created_utc < end_time:  # Check if comment is within time range
                comment_list.append(comment.body)
    except:
        print("Account not valid (probably suspended).")
        
    if len(comment_list) <= thresh:
        return None
        
    # This function deals with reformatting of the comment-array input.

    # Concatinates all the comments into one sample of the Redditor's writing (string object).
    writing_sample = ' '.join(comment_list)
    
    # Remove new line
    writing_sample = writing_sample.replace('\n', " ")

    # Removing all non-ASCII letters
    writing_sample = writing_sample.encode("ascii", "ignore").decode()
    
    # Remove URLs from comments
    writing_sample = re.sub(r'\(https?:\/\/.*?\)', '', writing_sample)

    # Now, we need to check for instances of elipses and assign them to a new symbol--less confusion with periods, easier analysis.
    # The tilde will represent an elipse from here on out.
    writing_sample = writing_sample.replace('...', '~')

    # We will loop over every character in the string and extract the punctuation, appending each mark to a list.
    # We will keep this punctuation sequence in both list and string form for further analysis.
    marks = ['.',',',':',';','?','!','-','(',')','"',"'",'~']
    marks_str = ''.join(map(str, marks))

    punct_list = []

    for character in writing_sample:
        if character in marks:
            punct_list.append(character)

    punct_str = ''.join(punct_list)
    
    if len(punct_list) <= thresh:
        return None

    # Now, we want to get our input in the following format..[2, '!', 3, '.'].
    split_by_punc = (x.split() for x in re.split('[' + marks_str + ']', writing_sample))
    wrds_btwn = list(map(len, split_by_punc))
    woven = list(chain.from_iterable(zip(wrds_btwn, punct_list)))
    
    # Develop punctuation stats for comments_text
    # Generation of F1, relative frequency of each punctuation mark of interest.
    num_period = punct_str.count('.')
    num_comma = punct_str.count(',')
    num_colon = punct_str.count(':')
    num_semicol = punct_str.count(';')
    num_question = punct_str.count('?')
    num_exclaim = punct_str.count('!')
    num_dash = punct_str.count('-')
    num_left_paren = punct_str.count('(')
    num_right_paren = punct_str.count(')')
    num_sing_quote = punct_str.count("'")
    num_doub_quote = punct_str.count('"')
    num_elipse = punct_str.count('~')

    mark_counts = [num_period, num_comma, num_colon, num_semicol, num_question, num_exclaim, num_dash, num_left_paren, num_right_paren, num_sing_quote, num_doub_quote, num_elipse] 
    F1 = [x / len(punct_list) for x in mark_counts]
    
    # Generation of F2 and F3, conditional/joint probability of observing two punctuation marks in succession.
    F2_dict = {}
    F3_dict = {}
    successive_pairs = []
    
    combos = list(itertools.product(marks_str, marks_str))
    indices = list(range(len(marks_str)))
    index_combos = list((itertools.product(indices, indices)))
    index_strings = [[str(x) for x in tup] for tup in index_combos]
    final_ind = list(map('-'.join, index_strings))
    #combos = list(itertools.product(marks_str, marks_str))
    #indices = list(range(marks_str)) 
    #index_combos = list(itertools.product(indices, indices)) 

    for i in combos:
        successive_pairs.append(''.join(i))
        
    zipped = list(zip(final_ind, successive_pairs)) 

    for (index, pair) in zipped:
        numerator = punct_str.count(pair)
        denominator = punct_str.count(pair[0])

        if numerator != 0:
            F2_dict['Conditional Prob' + ' ' + index] = (numerator/denominator)
            F3_dict['Joint Prob' + ' ' + index] = (numerator/len(punct_list))
            #F2_dict['Conditional Prob' + ' ' + index + ' ' + pair + ' '] = (numerator/denominator)
            #F3_dict['Joint Prob' + ' ' + index + ' ' + pair + ' '] = (numerator/len(punct_list))
        else:
            F2_dict['Conditional Prob' + ' ' + index] = 0
            F3_dict['Joint Prob' + ' ' + index] = 0
            #F2_dict['Conditional Prob' + ' ' + index + ' ' + pair + ' '] = 0
            #F3_dict['Joint Prob' + ' ' + index + ' ' + pair + ' '] = 0
            
    # Generation of f4, redditor's average sentence length in words.
    sent_lens = []
    max_sents = 200
    regex = "\w+('\w+)?(?<!('s))"
    sentences = split_into_sentences(writing_sample)

    for sentence in sentences:
        sent_lens.append(len(re.findall(regex, sentence)))

    if len(sent_lens) == 0:
        sent_lens.append(0)
    
    f4 = mean(sent_lens)

    # f4 as a probability distribution of sentence length--as it is in Darmon's work.
    # Cap sentence length at 199 words.
    F4 = [0]*max_sents
    
    sent_lens = [x if x < max_sents else max_sents-1 for x in sent_lens]

    for i in sent_lens:
        F4[i] += 1

    F4 = [x / len(sent_lens) for x in F4]
    
    # Generation of f5, redditor's average number of words between successive punctuation marks.
    if len(wrds_btwn) == 0:
        wrds_btwn.append(0)
        
    f5 = mean(wrds_btwn)

    # f5 as a probability distribution of number of words between successive marks--as it is in Darmon's work.
    # Cap number of in-between words at 39.
    max_wrds_btwn = 40
    wrds_btwn = [x if x < max_wrds_btwn else max_wrds_btwn-1 for x in wrds_btwn]
    F5 = [0]*max_wrds_btwn

    for i in wrds_btwn:
        F5[i] += 1

    F5 = [x / len(wrds_btwn) for x in F5]
    
    # Generation of f6, ratio of punctuation to words.
    total_punc = len(punct_list)
    total_words = len(re.findall(regex, writing_sample))

    f6 = total_punc/total_words
    
    # Output to dataframe.
    # An uppercase F signals that the feature was mimicked directly from Darmon's work.
    F1_dict_keys = ['Period Freq', 'Comma Freq', 'Colon Freq', 'Semicol Freq', 'Question Freq', 'Exclaim Freq', 'Dash Freq', 'L. Paren Freq', 'R. Paren Freq', 'S. Quote Freq', 'D. Quote Freq', 'Elipse Freq']
    F1_dict = {F1_dict_keys[i]: F1[i] for i in range(len(F1_dict_keys))} 

    F4_dict = {'Prob Sentence Len:' + ' ' + str(i): F4[i] for i in range(len(F4))}
    F5_dict = {'Prob' + ' ' + str(i) + ' ' + 'Words Between Punc': F5[i] for i in range(len(F5))}

    f4_dict = {'Avg Sentence Len': f4}
    f5_dict = {'Avg Wrds Btwn Punc': f5}
    f6_dict = {'Punc Ratio': f6}
    
    # Concatinate all the keys into a single list.
    master = {**F1_dict, **F2_dict, **F3_dict, **F4_dict, **F5_dict, **f4_dict, **f5_dict, **f6_dict}
    
    return master

In [6]:
# Helper function which splits text input into individual sentances. Used in generation of f4.

acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
alphabets= "([A-Za-z])"
digits = "([0-9])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text) 
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences